In [1]:
import requests
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from glob import glob
from tqdm import tqdm
import time
import json
import os
import gc

In [2]:
url = 'https://b.cari.com.my/forum.php'

In [3]:
r = requests.get(url)
soup = BeautifulSoup(r.content, "lxml")

In [4]:
a = soup.find_all('a')
a = [a_.get('href') for a_ in a if a_.get('href')]
a = [a_ for a_ in a if 'forum.php?mod=forumdisplay&fid=' in a_]
forums = list(set(a))

In [5]:
with open('forums.json', 'w') as fopen:
    json.dump(forums, fopen)

In [19]:
def get_href(url, forum):
    while True:
        try:
            r = requests.get(url, timeout = 5.0)
            break
        except:
            time.sleep(1.0)
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href')]
    a = [a_ for a_ in a if forum in a_ and '&page=' in a_[-10:]]
    try:
        max_page = max([int(a_.split('page=')[-1]) for a_ in a])
    except:
        max_page = -1
    
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href')]
    a = [a_.split('&extra=')[0] for a_ in a if a_.startswith('forum.php?mod=viewthread&tid=')]
    a = list(set(a))
        
    return a, max_page

In [20]:
forums[0]

'forum.php?mod=forumdisplay&fid=60'

In [21]:
url

'https://b.cari.com.my/forum.php'

In [22]:
forum = 'forum.php?mod=forumdisplay&fid=74'
filtered, max_page = get_href('https://b.cari.com.my/' + forum, forum)
filtered, max_page

(['forum.php?mod=viewthread&tid=1192930',
  'forum.php?mod=viewthread&tid=5166783',
  'forum.php?mod=viewthread&tid=773605',
  'forum.php?mod=viewthread&tid=5177025',
  'forum.php?mod=viewthread&tid=5176870',
  'forum.php?mod=viewthread&tid=803850',
  'forum.php?mod=viewthread&tid=1242793',
  'forum.php?mod=viewthread&tid=1190090',
  'forum.php?mod=viewthread&tid=1161301',
  'forum.php?mod=viewthread&tid=5175798',
  'forum.php?mod=viewthread&tid=1087706',
  'forum.php?mod=viewthread&tid=740483',
  'forum.php?mod=viewthread&tid=755667',
  'forum.php?mod=viewthread&tid=1029495',
  'forum.php?mod=viewthread&tid=1034686',
  'forum.php?mod=viewthread&tid=1256246',
  'forum.php?mod=viewthread&tid=5156681',
  'forum.php?mod=viewthread&tid=1212010',
  'forum.php?mod=viewthread&tid=1257202',
  'forum.php?mod=viewthread&tid=1219899',
  'forum.php?mod=viewthread&tid=5176367',
  'forum.php?mod=viewthread&tid=5048090',
  'forum.php?mod=viewthread&tid=1161526',
  'forum.php?mod=viewthread&tid=505610

In [24]:
!rm -rf forums
!mkdir forums

In [1]:
max_worker = 1

forums = sorted(forums)
for i in tqdm(range(len(forums))):
    filename = os.path.join('forums', f'{i}.json')
    if os.path.exists(filename) and os.path.getsize(filename) > 1000:
        continue
        
    forum = forums[i]
    url = 'https://b.cari.com.my/' + forum
    filtered, max_page = get_href(url, forum)
    for i in tqdm(range(2, max_page + 1, max_worker)):
        aranged = np.arange(i, i + max_worker)
        urls = [(f'{url}&page={a}', forum) for a in aranged]
        
        with ThreadPoolExecutor(max_workers=max_worker) as executor:
            futures = {executor.submit(get_href, url[0], url[1]): url for url in urls}

            for future in as_completed(futures):
                filtered.extend(future.result()[0])
                
    with open(filename, 'w') as fopen:
        json.dump(filtered, fopen)

NameError: name 'forums' is not defined